In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2.3.0
2.4.3


In [7]:
train = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/train.csv.zip', compression='zip', header=0, sep=',', quotechar='"')
test = pd.read_csv('../input/jigsaw-toxic-comment-classification-challenge/test.csv.zip', compression='zip', header=0, sep=',', quotechar='"')

In [8]:
test.head()



,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras import initializers, regularizers, constraints, optimizers, layers

In [10]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_train = train["comment_text"]
list_sentences_test = test["comment_text"]

In [11]:
#y1=test[list_classes].values
test.head

<bound method NDFrame.head of                       id                                       comment_text
0       00001cee341fdb12  Yo bitch Ja Rule is more succesful then you'll...
1       0000247867823ef7  == From RfC == \n\n The title is fine as it is...
2       00013b17ad220c46  " \n\n == Sources == \n\n * Zawe Ashton on Lap...
3       00017563c3f7919a  :If you have a look back at the source, the in...
4       00017695ad8997eb          I don't anonymously edit articles at all.
...                  ...                                                ...
153159  fffcd0960ee309b5  . \n i totally agree, this stuff is nothing bu...
153160  fffd7a9a6eb32c16  == Throw from out field to home plate. == \n\n...
153161  fffda9e8d6fafa9e  " \n\n == Okinotorishima categories == \n\n I ...
153162  fffe8f1340a79fc2  " \n\n == ""One of the founding nations of the...
153163  ffffce3fb183ee80  " \n :::Stop already. Your bullshit is not wel...

[153164 rows x 2 columns]>

In [12]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [13]:
maxlen = 200
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_tokenized_test, maxlen=maxlen) #Test set not used rn

In [14]:
#Using .add
model = Sequential()
model.add(Embedding(20000, 128, input_length=200) )
model.add(LSTM(60,return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(6, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', 
                           metrics=['accuracy']) 
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 128)          2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 200, 60)           45360     
_________________________________________________________________
global_max_pooling1d (Global (None, 60)                0         
_________________________________________________________________
dropout (Dropout)            (None, 60)                0         
_________________________________________________________________
dense (Dense)                (None, 50)                3050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 3

In [15]:
batch_size = 32
epochs = 4
model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/4
4488/4488 [==============================] - 164s 37ms/step - loss: 0.0679 - accuracy: 0.9440 - val_loss: 0.0494 - val_accuracy: 0.9940
Epoch 2/4
4488/4488 [==============================] - 165s 37ms/step - loss: 0.0444 - accuracy: 0.9896 - val_loss: 0.0473 - val_accuracy: 0.9937
Epoch 3/4
4488/4488 [==============================] - 165s 37ms/step - loss: 0.0390 - accuracy: 0.9832 - val_loss: 0.0472 - val_accuracy: 0.9936
Epoch 4/4
4488/4488 [==============================] - 163s 36ms/step - loss: 0.0346 - accuracy: 0.9706 - val_loss: 0.0481 - val_accuracy: 0.9853


In [16]:
model.save('model.h5')

In [17]:
#Checking
sent=["Nice amazing kind person"]
hypo = tokenizer.texts_to_sequences(sent)
hypo = pad_sequences(hypo, maxlen=200)
score=model.predict(hypo)
print (score)

[[5.5467174e-03 4.6733794e-06 5.2367308e-04 6.3040629e-06 6.6380831e-04
  2.0372527e-05]]
